# Import dependencies and prepare default directory

In [ ]:
  import vlab4mic
  import numpy as np
  import matplotlib.pyplot as plt
from vlab4mic import experiments

# Define structure and modalities to use

In [ ]:
structure = "7R5K"
modalities = ["STED", "SMLM"]

In [ ]:
labelling_efficiency = 0.01
random_orientations = False
number_of_particles = 10

In [ ]:
outputs, experiment = experiments.image_vsample(
    structure = structure, 
    multimodal = modalities, 
    labelling_efficiency = labelling_efficiency, 
    random_orientations = random_orientations, 
    number_of_particles = number_of_particles)

# View output images

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
nmods = len(modalities)

fig, axs = plt.subplots(1, nmods)
nframe = 0
for i, mod in enumerate(modalities):
    axs[i].imshow(outputs[mod][nframe], cmap="gray")
experiment.imager.show_field()